In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/home/antoine/homhots/HOTS/HOTS


In [2]:
import os
import pickle
import datetime
from mix_Network import *

dataset = 'nmnist'
path = f'../Records/{dataset}/models/'
timestr = '2021-02-16'

%mkdir -p ../Records
#%mkdir -p ../Records/EXP_03_NMNIST

sigma = None
pooling = False
homeinv = False
jitonic = [None,None] #[temporal, spatial]
jitter = False
tau = 5
nb_train = 5000
nb_test = 100
nblay = 3
nbclust = 4
filt = 2

ds = 50
nb_train_b = nb_train//ds
nb_test_b = nb_test//ds

In [3]:
def netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter):
    if name=='hots':
        homeo = False
        homeotest = False
        krnlinit = 'first'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learning1by1()
    elif name=='homhots':
        homeo = True
        homeotest = True
        krnlinit = 'rdn'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learningall()
    elif name=='onlyonline':
        homeo = False
        homeotest = False
        krnlinit = 'rdn'
        hotshom = network(krnlinit=krnlinit, filt=filt, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
        hotshom = hotshom.learningall()
    return hotshom, homeotest

In [4]:
def get_nmnist(NbTrainingData, train):
    def loadev(nbdigit, loader, learningset):
        timout = []
        xout = []
        yout = []
        polout = []
        labout = []
        for i in range(NbTrainingData):
            events, target = next(iter(loader))
            for iev in range(events.shape[1]):
                    timout.append(events[0][iev][learningset.ordering.find("t")].item())
                    xout.append(events[0][iev][learningset.ordering.find("x")].item())
                    yout.append(events[0][iev][learningset.ordering.find("y")].item())
                    polout.append(events[0][iev][learningset.ordering.find("p")].item())
                    labout.append(target.item())

        eventsout = [xout,yout,timout,polout,labout,learningset.sensor_size,2]
        return eventsout
    
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                    train=train,
                                    transform=None)
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    events_train = loadev(NbTrainingData, loader, learningset)
    
    return events_train


In [5]:
#### Building matrix for logistic regression
def gather_data(events_in,
                tau_cla=150, # characteristic time of a digit
                sample_events=50, sample_space = 1,
                verbose=False, debug=False):
    tau_cla *= 1e3 # to enter tau in ms
    n_events = len(events_in[0])

    c_int = lambda n, d : ((n - 1) // d) + 1
    
    data = np.zeros((c_int(events_in[-2][0], sample_space),
                     c_int(events_in[-2][1], sample_space),
                     events_in[-1])) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):
        if events_in[2][i_event]<events_in[2][i_event-1]:
            data = np.zeros((c_int(events_in[-2][0], sample_space),
                     c_int(events_in[-2][1], sample_space),
                     events_in[-1])) #tmp data 

        data *= np.exp(-(events_in[2][i_event]-events_in[2][i_event-1])/tau_cla)
        
        x_pos = events_in[0][i_event]//sample_space
        y_pos = events_in[1][i_event]//sample_space
        p = events_in[3][i_event]
        data[int(x_pos), int(y_pos), int(p)] = 1.

        if i_event % sample_events == sample_events//2 :
            if debug:
                print(f'DEBUG {i_event} {i_event//sample_events} ')
                print(f'DEBUG {y[i_event//sample_events]}   ')
                print(f'DEBUG  {events_in[-3][i_event]} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = events_in[-3][i_event]
            
    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))
    
#    y_out = np.zeros([y.shape[0],int(max(y)+1)])
#    for i in range(y.shape[0]):
#        y_out[i,int(y[i])]=1
    
    return X, y

In [6]:
# from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LR
# 
opts_LR = dict(max_iter=2000, # random_state=0,
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
opts_LR['Cs'] = 32
# TODO for a publication use 100 from 10^{-10} to 10

from sklearn.model_selection import train_test_split

def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')


# Performing logistic regression on raw input spikes

# Performing logistic regression on the network output
## With homeostasis

In [7]:
name = 'homhots'
hotshom, homeotest = netparam(name, filt, tau, nblay, nbclust, sigma, homeinv, jitter)
print('model loaded')
events_train = hotshom.running(homeotest = homeotest, nb_digit = nb_train, LR=True)

2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False
model loaded
../Records/nmnist/train/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_5000_[None, None]_homeo.pkl


In [8]:
train = True
tic()
nb_evt = len(events_train[0])
lr = LR(**opts_LR)
for batch in range(ds):
    events_train_b = []
    for i in range(5):
        events_train_b.append(events_train[i][batch*nb_evt//ds:(batch+1)*nb_evt//ds])
    for i in range(5,len(events_train)):
        events_train_b.append(events_train[i])
    X_train, y_train = gather_data(events_train_b, verbose=True)
    lr.fit(X_train, y_train)
    print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f} (train)')
toc()
f_name = path+hotshom.name+f'_LR_{nb_train}.pkl'
with open(f_name, 'wb') as file:
    pickle.dump(lr, file, pickle.HIGHEST_PROTOCOL)

Number of events: 7810 - Number of features: 18496
Classification score for raw input is 0.999 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 1.000 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 0.999 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 0.999 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 0.998 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 1.000 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 1.000 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 0.999 (train)
Number of events: 7810 - Number of features: 18496
Classification score for raw input is 0.999 (train)
Number of events: 7810 - Number of features: 18496
Classification score f

In [9]:
train = False
events_test = hotshom.running(homeotest = homeotest, nb_digit = nb_test_b, train=False, LR=True)

tic()
X_test, y_test = gather_data(events_test, verbose=True)
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f} (test)')
toc()

../Records/nmnist/test/2021-02-16_lagorce_rdn_None_True_[0.25, 1]_[4, 8, 16]_[5000.0, 50000.0, 500000.0]_[2, 4, 8]_False_2_[None, None]_homeo.pkl
Number of events: 118 - Number of features: 18496
Classification score for raw input is 0.364 (test)
Done in 0.061 s
